In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# where we'll save charts for GitHub
FIG_DIR = "../reports/figures"
os.makedirs(FIG_DIR, exist_ok=True)

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

In [5]:
df = pd.read_csv("../data/raw/events.csv")
df["event_time"] = pd.to_datetime(df["event_time"])

df.head()

,user_id,event_time,event_name,channel,country,device,campaign,deposit_amount,trade_volume
0,1,2024-03-12 20:38:00,visit,organic,FR,mobile,NaN,NaN,NaN
1,1,2024-03-12 21:05:00,sign_up,organic,FR,mobile,NaN,NaN,NaN
2,1,2024-03-12 22:37:00,kyc_start,organic,FR,mobile,NaN,NaN,NaN
3,1,2024-03-14 14:37:00,kyc_approved,organic,FR,mobile,NaN,NaN,NaN
4,2,2024-01-30 05:01:00,visit,organic,GR,desktop,NaN,NaN,NaN


In [4]:
df.shape, df["event_name"].value_counts()

((23261, 9),
 event_name
 visit            10000
 sign_up           4539
 kyc_start         3419
 kyc_approved      2582
 first_deposit     1295
 first_trade       1029
 retained_7d        397
 Name: count, dtype: int64)

In [6]:
STAGES = ["visit", "sign_up", "kyc_start", "kyc_approved", "first_deposit", "first_trade", "retained_7d"]

funnel = (
    df[df["event_name"].isin(STAGES)]
    .groupby("event_name")["user_id"]
    .nunique()
    .reindex(STAGES)
    .rename("users")
    .to_frame()
)

funnel["overall_conv_from_visit"] = funnel["users"] / funnel.loc["visit", "users"]
funnel["step_conv_from_prev"] = funnel["users"] / funnel["users"].shift(1)
funnel["step_dropoff"] = 1 - funnel["step_conv_from_prev"]

funnel


,users,overall_conv_from_visit,step_conv_from_prev,step_dropoff
event_name,,,,
visit,10000,1.0000,NaN,NaN
sign_up,4539,0.4539,0.453900,0.546100
kyc_start,3419,0.3419,0.753250,0.246750
kyc_approved,2582,0.2582,0.755192,0.244808
first_deposit,1295,0.1295,0.501549,0.498451
first_trade,1029,0.1029,0.794595,0.205405
retained_7d,397,0.0397,0.385811,0.614189


In [12]:
FIG_DIR = "../reports/figures"
os.makedirs(FIG_DIR, exist_ok=True)

plt.figure()
plt.bar(funnel.index, funnel["users"])
plt.xticks(rotation=45, ha="right")
plt.title("Forex App Funnel — Unique Users by Stage")
plt.ylabel("Users")
plt.tight_layout()

out_path = f"{FIG_DIR}/01_funnel_users_by_stage.png"
plt.savefig(out_path, dpi=200)
plt.show()

out_path


'../reports/figures/01_funnel_users_by_stage.png'

In [13]:
plt.figure()
plt.plot(funnel.index, funnel["step_conv_from_prev"], marker="o")
plt.xticks(rotation=45, ha="right")
plt.title("Step Conversion Rate (Stage → Next Stage)")
plt.ylabel("Conversion rate")
plt.ylim(0, 1.05)
plt.tight_layout()

out_path = f"{FIG_DIR}/02_step_conversion_rates.png"
plt.savefig(out_path, dpi=200)
plt.show()

out_path


'../reports/figures/02_step_conversion_rates.png'

In [15]:
channel_tbl = (
    df[df["event_name"].isin(STAGES)]
    .pivot_table(index="channel", columns="event_name", values="user_id", aggfunc=pd.Series.nunique)
    .reindex(columns=STAGES)
    .fillna(0)
    .astype(int)
)

channel_tbl["visit_to_signup"] = channel_tbl["sign_up"] / channel_tbl["visit"]
channel_tbl["signup_to_kyc_approved"] = channel_tbl["kyc_approved"] / channel_tbl["sign_up"]
channel_tbl["kyc_to_deposit"] = channel_tbl["first_deposit"] / channel_tbl["kyc_approved"]
channel_tbl["deposit_to_trade"] = channel_tbl["first_trade"] / channel_tbl["first_deposit"]
channel_tbl["trade_to_retained_7d"] = channel_tbl["retained_7d"] / channel_tbl["first_trade"]

channel_tbl.sort_values("first_trade", ascending=False)

event_name,visit,sign_up,kyc_start,kyc_approved,first_deposit,first_trade,retained_7d,visit_to_signup,signup_to_kyc_approved,kyc_to_deposit,deposit_to_trade,trade_to_retained_7d
channel,,,,,,,,,,,,
paid,3542,1610,1193,920,537,432,176,0.454545,0.571429,0.583696,0.804469,0.407407
organic,3935,1794,1385,1038,440,351,124,0.455909,0.578595,0.423892,0.797727,0.353276
referral,1528,695,526,387,178,145,52,0.454843,0.556835,0.459948,0.814607,0.358621
affiliate,995,440,315,237,140,101,45,0.442211,0.538636,0.590717,0.721429,0.445545


In [17]:
plot_df = channel_tbl[["first_trade", "retained_7d"]].sort_values("first_trade", ascending=False)

plt.figure()
x = np.arange(len(plot_df.index))
width = 0.4

plt.bar(x - width/2, plot_df["first_trade"], width, label="first_trade users")
plt.bar(x + width/2, plot_df["retained_7d"], width, label="retained_7d users")

plt.xticks(x, plot_df.index)
plt.title("Channel Performance — First Trade vs 7-day Retention (Users)")
plt.ylabel("Users")
plt.legend()
plt.tight_layout()

out_path = f"{FIG_DIR}/03_channel_trade_vs_retention.png"
plt.savefig(out_path, dpi=200)
plt.show()

out_path

'../reports/figures/03_channel_trade_vs_retention.png'